<a href="https://colab.research.google.com/github/Zahra-Ferdous/Thesis_Human_Action_Recognition_from_Depth_Video/blob/main/local_ternary_pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn
import math
%matplotlib inline
import os
import time

from numba.core.cgutils import if_likely
#from os.path import exists
import os.path 

import cv2
import sklearn
import scipy.io as sio
from numba import jit
from math import factorial, log
from sklearn.neighbors import KDTree
from scipy.signal import periodogram, welch
from astropy.timeseries import LombScargle
from scipy.io import loadmat


import imageio
from skimage import feature

In [8]:
def get_dmm(d):
  dep, ht, wd = d.shape
  F = np.zeros((ht, wd))

  for i in range(dep):
    
      front = d[i,:,:]

      if(i>0):
        F = np.add( F, np.absolute( np.subtract( front, front_pre ) ) )

      front_pre = front

  return F

In [6]:
""" 
Center Symmetric Local Ternary Pattern 
Code might contain bugs
"""

from tqdm import tqdm
import numpy as np

def sigmoid(x):

	if x > 3:
		return 2
	elif x < -3:
		return 1
	else:
		return 0 

def get_csLTP(images, img_height, img_width):

	zeroHorizontal = np.zeros(img_width + 2).reshape(1, img_width + 2)
	zeroVertical = np.zeros(img_height).reshape(img_height, 1)

	features = []

	for img in tqdm(images):

		img = np.concatenate((img, zeroVertical), axis = 1)
		img = np.concatenate((zeroVertical, img), axis = 1)
		img = np.concatenate((zeroHorizontal, img), axis = 0)
		img = np.concatenate((img, zeroHorizontal), axis = 0)

		pattern_img = np.zeros((img_height + 1, img_width + 1))
		
		for x in range(1, img_height + 1):
			for y in range(1, img_width + 1):
				
				s1 = sigmoid(img[x-2, y-2] - img[x+2, y+2])
				s3 = sigmoid(img[x-2, y+2] - img[x+2, y-2])*3
		
				s = s1 + s3
			
				pattern_img[x, y] = s

		pattern_img = pattern_img[1:(img_height+1), 1:(img_width+1)].astype(int) 	
		histogram = np.histogram(pattern_img, bins = np.arange(17))[0]
		histogram = histogram.reshape(1, -1)

		features.append(histogram)

	features = np.concatenate(features, axis = 0)

	return features

In [21]:
""" Extended Center Symmetric Local Ternary Pattern """

from tqdm import tqdm
import numpy as np

def heaviside(x, y):
	
	if x > 3 and y > 3:
		return 2
	elif x < -3 and y < -3:
		return 1
	else:
		return 0

def get_xcsLTP(images, img_height, img_width):

	zeroHorizontal = np.zeros(img_width + 2).reshape(1, img_width + 2)
	zeroVertical = np.zeros(img_height).reshape(img_height, 1)

	features = []

	for img in tqdm(images):

		img = np.concatenate((img, zeroVertical), axis = 1)
		img = np.concatenate((zeroVertical, img), axis = 1)
		img = np.concatenate((zeroHorizontal, img), axis = 0)
		img = np.concatenate((img, zeroHorizontal), axis = 0)

		pattern_img = np.zeros((img_height + 1, img_width + 1))
		
		for x in range(1, img_height + 1):
			for y in range(1, img_width + 1): 

				s1 = heaviside(img[x-1, y-1]-img[x, y], img[x, y]-img[x+1, y+1]) 
				s3 = heaviside(img[x-1, y+1]-img[x, y], img[x, y]-img[x+1, y-1])*3
				
				s = s1 + s3

				pattern_img[x, y] = s

		pattern_img = pattern_img[1:(img_height+1), 1:(img_width+1)].astype(int) 		
		histogram = np.histogram(pattern_img, bins = np.arange(17))[0]
		histogram = histogram.reshape(1, -1)

		features.append(histogram)

	features = np.concatenate(features, axis = 0)

	return features

In [ ]:
# def get_ltp(img):
#   zeroHorizontal = np.zeros(img.shape[1] + 2).reshape(1, img.shape[1] + 2)
# 	zeroVertical = np.zeros(img.shape[0]).reshape(img.shape[0], 1)
 

In [22]:
path = '/content/drive/MyDrive/Colab Notebooks Copy Old/utd mhad depth/Depth'

cnt = 0
cmt = 0

f_dmm = []

for file in os.listdir(path):
    if file.endswith('.mat'):
      cnt+=1
      if(cnt==12):
        break
      d = sio.loadmat(path+'/'+file)['d_depth']
      d = np.transpose(d, (2, 0, 1))
      print(d.shape)
      front = get_dmm(d)
      print(front.shape)
      f_dmm.append(front)
      
ltp = get_xcsLTP(f_dmm, front.shape[0],front.shape[1])
print(ltp.shape)


(48, 240, 320)
(240, 320)
(52, 240, 320)
(240, 320)
(53, 240, 320)
(240, 320)
(61, 240, 320)
(240, 320)
(61, 240, 320)
(240, 320)
(54, 240, 320)
(240, 320)
(54, 240, 320)
(240, 320)
(55, 240, 320)
(240, 320)
(72, 240, 320)
(240, 320)
(72, 240, 320)
(240, 320)
(69, 240, 320)
(240, 320)


100%|██████████| 11/11 [00:02<00:00,  3.95it/s]

(11, 16)


In [23]:
print(ltp)

[[72896   632   648   649   176   382   770   488   159     0     0     0
      0     0     0     0]
 [72909   624   589   712   172   414   739   460   181     0     0     0
      0     0     0     0]
 [73056   628   593   604   163   410   712   451   183     0     0     0
      0     0     0     0]
 [72797   681   633   690   149   412   797   487   154     0     0     0
      0     0     0     0]
 [72828   664   678   629   186   457   712   478   168     0     0     0
      0     0     0     0]
 [72782   709   663   680   179   495   703   443   146     0     0     0
      0     0     0     0]
 [72856   701   639   706   181   431   682   449   155     0     0     0
      0     0     0     0]
 [72853   686   682   649   214   448   666   437   165     0     0     0
      0     0     0     0]
 [72363   703   764   743   217   548   703   559   200     0     0     0
      0     0     0     0]
 [72341   728   745   770   230   475   734   557   220     0     0     0
      0     0    